In [10]:
from abc import ABC, abstractmethod
from typing import Iterable, Mapping, Sequence

import pandas as pd

from nepal.datasets import Dataset, NYTimes, PopulationDensity, GovernmentResponse, Vaccinations

In [2]:
df = GovernmentResponse().load()

In [8]:
df_usa = df[(df["CountryCode"] == "USA") & (df["RegionCode"].notna())]

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_combined_numeric,C1_combined,C2_combined_numeric,C2_combined,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
243474,United States,USA,Alaska,US_AK,STATE_TOTAL,2020-01-01,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243475,United States,USA,Alaska,US_AK,STATE_TOTAL,2020-01-02,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243476,United States,USA,Alaska,US_AK,STATE_TOTAL,2020-01-03,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243477,United States,USA,Alaska,US_AK,STATE_TOTAL,2020-01-04,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243478,United States,USA,Alaska,US_AK,STATE_TOTAL,2020-01-05,0.0,0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288043,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-19,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288044,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-20,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288045,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-21,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288046,United States,USA,Wyoming,US_WY,STATE_TOTAL,2022-05-22,NaN,<NA>,NaN,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_left = df_usa[
    [
        "RegionCode",
        "Date",
        "StringencyIndex",
        "GovernmentResponseIndex",
        "ContainmentHealthIndex",
        "EconomicSupportIndex",
    ]
].set_index(["RegionCode", "Date"])

In [21]:
df = Vaccinations().load()
df = df[df["FIPS"] != "UNK"]

{'AK',
 'AL',
 'AR',
 'AZ',
 'CA',
 'CO',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'GU',
 'HI',
 'IA',
 'ID',
 'IL',
 'IN',
 'KS',
 'KY',
 'LA',
 'MA',
 'MD',
 'ME',
 'MI',
 'MN',
 'MO',
 'MS',
 'MT',
 'NC',
 'ND',
 'NE',
 'NH',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'OK',
 'OR',
 'PA',
 'PR',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VA',
 'VI',
 'VT',
 'WA',
 'WI',
 'WV',
 'WY'}

In [39]:
import numpy as np


def _add_derived_columns(data: pd.DataFrame) -> pd.DataFrame:
    data["RegionCode"] = ("US_" + data["Recip_State"]).astype("string")
    data["Under5_Pop_Pct"] = (data["Census2019"] - data["Census2019_5PlusPop"]) / data[
        "Census2019"
    ]
    data["Between5to17_Pop_Pct"] = data["Census2019_5to17Pop"] / data["Census2019"]
    data["Between18to65_Pop_Pct"] = (
        data["Census2019_18PlusPop"] - data["Census2019_65PlusPop"]
    ) / data["Census2019"]
    data["Plus65_Pop_Pct"] = data["Census2019_65PlusPop"] / data["Census2019"]
    data["Is_Metro"] = np.where(data["Metro_status"] == "Metro", 1, 0)
    data["SVI_A"] = np.where(data["SVI_CTGY"] == "A", 1, 0)
    data["SVI_B"] = np.where(data["SVI_CTGY"] == "B", 1, 0)
    data["SVI_C"] = np.where(data["SVI_CTGY"] == "C", 1, 0)
    data["SVI_D"] = np.where(data["SVI_CTGY"] == "D", 1, 0)
    return data

In [42]:
df_ex = _add_derived_columns(df)

df_right = (
    df_ex[
        [
            "Date",
            "RegionCode",
            "FIPS",
            "Completeness_pct",
            "Administered_Dose1_Pop_Pct",
            "Administered_Dose1_Recip_18PlusPop_Pct",
            "Administered_Dose1_Recip_65PlusPop_Pct",
            "Series_Complete_Pop_Pct",
            "Series_Complete_18PlusPop_Pct",
            "Series_Complete_65PlusPop_Pct",
            "Booster_Doses_Vax_Pct",
            "Booster_Doses_18Plus_Vax_Pct",
            "Booster_Doses_50Plus_Vax_Pct",
            "Booster_Doses_65Plus_Vax_Pct",
            "Under5_Pop_Pct",
            "Between5to17_Pop_Pct",
            "Between18to65_Pop_Pct",
            "Plus65_Pop_Pct",
            "Is_Metro",
            "SVI_A",
            "SVI_B",
            "SVI_C",
            "SVI_D",
        ]
    ]
    .set_index(["RegionCode", "Date"])
    .sort_index()
)

In [43]:
df_right

FIPS  Completeness_pct  Administered_Dose1_Pop_Pct  \
RegionCode Date                                                              
US_AK      2020-12-13  02130               0.0                         0.0   
           2020-12-13  02122               0.0                         0.0   
           2020-12-13  02090               0.0                         0.0   
           2020-12-13  02100               0.0                         0.0   
           2020-12-13  02170               0.0                         0.0   
...                      ...               ...                         ...   
US_WY      2022-05-21  56027         97.099998                   34.099998   
           2022-05-21  56021         97.099998                   64.699997   
           2022-05-21  56017         97.099998                   52.299999   
           2022-05-21  56037         97.099998                   59.200001   
           2022-05-21  56011         97.099998                        36.0   

                       Administered_Dose1_Recip_18PlusPop_Pct  \
RegionCode Date                                                 
US_AK      2020-12-13                                     0.0   
           2020-12-13                                     0.0   
           2020-12-13                                     0.0   
           2020-12-13                                     0.0   
           2020-12-13                                     0.0   
...                                                       ...   
US_WY      2022-05-21                               40.400002   
           2022-05-21                               76.800003   
           2022-05-21                               62.400002   
           2022-05-21                                    73.0   
           2022-05-21                                    45.5   

                       Administered_Dose1_Recip_65PlusPop_Pct  \
RegionCode Date                                                 
US_AK      2020-12-13                                     0.0   
           2020-12-13                                     0.0   
           2020-12-13                                     0.0   
           2020-12-13                                     0.0   
           2020-12-13                                     0.0   
...                                                       ...   
US_WY      2022-05-21                               67.099998   
           2022-05-21                                    95.0   
           2022-05-21                                    85.0   
           2022-05-21                                    95.0   
           2022-05-21                               73.699997   

                       Series_Complete_Pop_Pct  Series_Complete_18PlusPop_Pct  \
RegionCode Date                                                                 
US_AK      2020-12-13                      0.0                            0.0   
           2020-12-13                      0.0                            0.0   
           2020-12-13                      0.0                            0.0   
           2020-12-13                      0.0                            0.0   
           2020-12-13                      0.0                            0.0   
...                                        ...                            ...   
US_WY      2022-05-21                     31.5                           37.5   
           2022-05-21                56.799999                      67.400002   
           2022-05-21                49.799999                      59.400002   
           2022-05-21                50.299999                      62.099998   
           2022-05-21                     31.0                           39.5   

                       Series_Complete_65PlusPop_Pct  Booster_Doses_Vax_Pct  \
RegionCode Date                                                               
US_AK      2020-12-13                            0.0                   <NA>   
           2020-12-13     

In [47]:
df_joined = df_left.join(df_right, on=["RegionCode", "Date"])

In [62]:
df_joined[df_joined["FIPS"].notna()].reset_index().drop(columns="RegionCode").set_index(
    ["FIPS", "Date"]
).sort_index()

StringencyIndex  GovernmentResponseIndex  \
FIPS  Date                                                   
01001 2020-12-13            43.52                    47.40   
      2020-12-14            43.52                    47.40   
      2020-12-15            43.52                    48.65   
      2020-12-16            43.52                    48.65   
      2020-12-17            43.52                    48.65   
...                           ...                      ...   
56045 2022-05-17            27.78                      NaN   
      2022-05-18              NaN                      NaN   
      2022-05-19              NaN                      NaN   
      2022-05-20              NaN                      NaN   
      2022-05-21              NaN                      NaN   

                  ContainmentHealthIndex  EconomicSupportIndex  \
FIPS  Date                                                       
01001 2020-12-13                   48.81                  37.5   
      2020-12-14                   48.81                  37.5   
      2020-12-15                   50.24                  37.5   
      2020-12-16                   50.24                  37.5   
      2020-12-17                   50.24                  37.5   
...                                  ...                   ...   
56045 2022-05-17                   35.71                   NaN   
      2022-05-18                     NaN                   NaN   
      2022-05-19                     NaN                   NaN   
      2022-05-20                     NaN                   NaN   
      2022-05-21                     NaN                   NaN   

                  Completeness_pct  Administered_Dose1_Pop_Pct  \
FIPS  Date                                                       
01001 2020-12-13               0.0                         0.0   
      2020-12-14               0.0                         0.0   
      2020-12-15               0.0                         0.0   
      2020-12-16               0.0                         0.0   
      2020-12-17               0.0                         0.0   
...                            ...                         ...   
56045 2022-05-17         97.099998                   41.299999   
      2022-05-18         97.099998                   41.299999   
      2022-05-19         97.099998                   41.299999   
      2022-05-20         97.099998                   41.299999   
      2022-05-21         97.099998                   41.299999   

                  Administered_Dose1_Recip_18PlusPop_Pct  \
FIPS  Date                                                 
01001 2020-12-13                                     0.0   
      2020-12-14                                     0.0   
      2020-12-15                                     0.0   
      2020-12-16                                     0.0   
      2020-12-17                                     0.0   
...                                                  ...   
56045 2022-05-17                               49.599998   
      2022-05-18                               49.599998   
      2022-05-19                               49.599998   
      2022-05-20                               49.599998   
      2022-05-21                               49.599998   

                  Administered_Dose1_Recip_65PlusPop_Pct  \
FIPS  Date                                                 
01001 2020-12-13                                     0.0   
      2020-12-14                                     0.0   
      2020-12-15                                     0.0   
      2020-12-16                                     0.0   
      2020-12-17                                     0.0   
...                                                  ...   
56045 2022-05-17                               76.199997   
      2022-05-18                               76.199997   
      2022-05-19                               76.199997   
      2022-05-20                               76.19